Como aún no tenemos datos etiquetados, lo primero que haremos es generar un pequeño conjunto de reseñas con etiquetas manuales.
Carga el dataset de reseñas desde un archivo CSV (reviews_gm_cleaned.csv).
Filtra las reseñas para eliminar aquellas cuyo texto es "No disponible".
Selecciona aleatoriamente un conjunto de reseñas (actualmente el usuario está ajustando el número de muestras).
Asigna etiquetas automáticamente a las reseñas según palabras clave detectadas en el texto. Las etiquetas posibles son:
"food" (comida, restaurante, menú, etc.).
"service" (servicio, atención al cliente, soporte, etc.).
"price" (precio, costoso, barato, etc.).
"location" (ubicación, vista, área, etc.).
Guarda el resultado en un nuevo CSV (reviews_ner_manual.csv) para revisión manual


In [8]:
import spacy
from spacy.training import Example
import pandas as pd
import random
import os

# 1. Cargar datasets con encoding UTF-8
path_reviews = r"D:\Soy Henry\Proyecto Final\reviews_gm_cleaned.csv"
path_metadata = r"D:\Soy Henry\Proyecto Final\metadata_clean.csv"

df_reviews = pd.read_csv(path_reviews, encoding="utf-8")
df_metadata = pd.read_csv(path_metadata, encoding="utf-8")

# 2. Filtrar reseñas donde el texto no sea "No disponible" y eliminar nulos
df_reviews = df_reviews[df_reviews["text"].notna() & (df_reviews["text"] != "No disponible")]

# 3. Unir datasets usando gmap_id
df_merged = df_reviews.merge(df_metadata, on="gmap_id", how="left")

# 4. Seleccionar 5000 reseñas aleatorias para etiquetar
sample_reviews = df_merged.sample(5000, random_state=42)

# 5. Diccionario de categorías y palabras clave
keywords = {
    "FOOD": {"food", "restaurant", "eat", "menu", "drink", "dish", "delicious", "taco", "boba", "meal", "dining", "cuisine", "flavor"},
    "SERVICE": {"service", "staff", "customer", "attention", "help", "support", "reception", "checkout", "waiter", "friendly", "rude"},
    "PRICE": {"price", "cost", "cheap", "expensive", "affordable", "overpriced", "value", "deal", "worth"},
    "LOCATION": {"location", "place", "area", "view", "scenery", "spot", "attraction", "neighborhood", "convenient", "access"}
}

def assign_entities(text):
    """
    Asigna entidades basadas en la ocurrencia de palabras clave.
    Devuelve un diccionario con el formato:
    {"entities": [(start_index, end_index, "LABEL"), ...]}
    """
    entities = []
    lower_text = text.lower()
    
    for label, words in keywords.items():
        for word in words:
            start = 0
            while start < len(lower_text):
                start = lower_text.find(word, start)
                if start == -1:
                    break
                end = start + len(word)
                
                # Verificar si la nueva entidad se solapa con alguna existente
                overlapping = False
                for entity in entities:
                    if not (end <= entity[0] or start >= entity[1]):  # Si se solapan
                        overlapping = True
                        break
                
                if not overlapping:
                    entities.append((start, end, label))
                
                start = end
    
    return {"entities": entities}

# 6. Preparar datos de entrenamiento en formato SpaCy
train_data = []
for _, row in sample_reviews.iterrows():
    text = row["text"]
    entities = assign_entities(text)
    train_data.append((text, entities))

# 7. Cargar un modelo preentrenado de SpaCy
nlp = spacy.load("en_core_web_md")  # Cambia "es" por "en" si estás trabajando con inglés

# 8. Convertir datos a formato SpaCy
examples = []
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)

# 9. Ajustar el modelo
optimizer = nlp.begin_training()
for epoch in range(10):  # Número de épocas
    random.shuffle(examples)
    for example in examples:
        nlp.update([example], drop=0.5, sgd=optimizer)

# 10. Guardar el modelo ajustado
output_model_path = r"D:\Soy Henry\Proyecto Final\yelp-google-reviews\Sprint_Nro3\Deployment_final_modelo_ML\adjusted_model"
os.makedirs(output_model_path, exist_ok=True)
nlp.to_disk(output_model_path)

print(f"✅ Modelo ajustado guardado en: {output_model_path}")


c:\Users\calde\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Great service!!" with entities "[(2, 5, 'FOOD'), (6, 13, 'SERVICE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\calde\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "This place is cozy, food is delicious, bar serves ..." with entities "[(58, 63, 'FOOD'), (20, 24, 'FOOD'), (28, 37, 'FOO...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
c:\Users\calde\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\training\iob_utils.py:149: Use

✅ Modelo ajustado guardado en: D:\Soy Henry\Proyecto Final\yelp-google-reviews\Sprint_Nro3\Deployment_final_modelo_ML\adjusted_model


In [13]:
import spacy

# Cargar el modelo entrenado desde la ruta proporcionada
output_model_path = r"D:\Soy Henry\Proyecto Final\yelp-google-reviews\Sprint_Nro3\Deployment_final_modelo_ML\adjusted_model"
nlp = spacy.load(output_model_path)

# Verificar las etiquetas de las entidades reconocidas por el modelo
print("Entidades que el modelo puede reconocer:")
for label in nlp.get_pipe("ner").labels:
    print(label)  # Imprime las etiquetas de entidades entrenadas

# También podemos verificar los componentes del pipeline
print("\nComponentes del pipeline:")
print(nlp.pipe_names)  # Imprime los nombres de los componentes del pipeline



Entidades que el modelo puede reconocer:
CARDINAL
DATE
EVENT
FAC
FOOD
GPE
LANGUAGE
LAW
LOC
LOCATION
MONEY
NORP
ORDINAL
ORG
PERCENT
PERSON
PRICE
PRODUCT
QUANTITY
SERVICE
TIME
WORK_OF_ART

Componentes del pipeline:
['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [14]:
# Texto de prueba con ejemplos que incluyen diversas entidades
texto_prueba = """
The Apple store in New York has the latest iPhone models. The new iPhone 15 is priced at $999 and it comes in various colors. 
The customer service was excellent, and I had a great experience. The location of the store is very convenient near Central Park.
I also love the food at the nearby restaurant, it’s amazing. The store hosts a special event every month.
"""

# Procesar el texto con el modelo NER
doc = nlp(texto_prueba)

# Mostrar las entidades reconocidas
for ent in doc.ents:
    print(f"Texto: {ent.text}, Etiqueta: {ent.label_}")


Texto: customer, Etiqueta: SERVICE
Texto: service, Etiqueta: SERVICE
Texto: location, Etiqueta: LOCATION
Texto: convenient, Etiqueta: LOCATION
Texto: food, Etiqueta: FOOD
Texto: restaurant, Etiqueta: FOOD


c:\Users\calde\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\pipeline\attributeruler.py:149: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  matches = self.matcher(doc, allow_missing=True, as_spans=False)
c:\Users\calde\AppData\Local\Programs\Python\Python312\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [15]:
# Ver las entidades reconocidas y su contexto
for ent in doc.ents:
    print(f"Texto: {ent.text}, Etiqueta: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")

# Ver las palabras tokenizadas y sus partes del discurso (POS)
for token in doc:
    print(f"Token: {token.text}, POS: {token.pos_}, Tag: {token.tag_}")


Texto: customer, Etiqueta: SERVICE, Start: 132, End: 140
Texto: service, Etiqueta: SERVICE, Start: 141, End: 148
Texto: location, Etiqueta: LOCATION, Start: 198, End: 206
Texto: convenient, Etiqueta: LOCATION, Start: 228, End: 238
Texto: food, Etiqueta: FOOD, Start: 274, End: 278
Texto: restaurant, Etiqueta: FOOD, Start: 293, End: 303
Token: 
, POS: , Tag: $
Token: The, POS: , Tag: $
Token: Apple, POS: , Tag: $
Token: store, POS: , Tag: $
Token: in, POS: , Tag: $
Token: New, POS: , Tag: $
Token: York, POS: , Tag: $
Token: has, POS: , Tag: $
Token: the, POS: , Tag: $
Token: latest, POS: , Tag: $
Token: iPhone, POS: , Tag: $
Token: models, POS: , Tag: $
Token: ., POS: , Tag: $
Token: The, POS: , Tag: $
Token: new, POS: , Tag: $
Token: iPhone, POS: , Tag: $
Token: 15, POS: , Tag: $
Token: is, POS: , Tag: $
Token: priced, POS: , Tag: $
Token: at, POS: , Tag: $
Token: $, POS: , Tag: $
Token: 999, POS: , Tag: $
Token: and, POS: , Tag: $
Token: it, POS: , Tag: $
Token: comes, POS: , Tag: $
To